<a href="https://colab.research.google.com/github/pragyansharma24/Gun-Violence-in-United-States/blob/master/Gun_Violence_in_the_United_States.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis of Gun Violence in the United States
#### Authored By: Pragyan Sharma, Prerana Patil, Shriya Rao, Sonal Agrawal

---

Over the years, there has been a continuous increase in the number of incidents related to gun violence in the United States. In addition to killing hundreds of Americans, it has forced them to live in a continuous fear of the next shooting. Therefore, in order to understand the magnitude of everyday gun violence, we have analyzed available data to understand:
<ul>
  <li>Crime Statistics across United States</li>
  <li>Crime Statistics across the World</li>
  <li>Presence of gun laws and death rates across United States  </li>
</ul>

## Crime Statistics across United States 

According to an article in New York Times, "*More people died from firearm injuries in the United States in 2017 than in any other year since at least 1968, according to new data from the Centers for Disease Control and Prevention*." (Ref: https://www.nytimes.com/2018/12/18/us/gun-deaths.html).

In fact, 2017 witnessed the highest number of mass shootings alone, killing and injuring hundreds of innocent lives. To understand the seriousness of the situation, we plan to visualize each of the mass shooting incidents on the map of the United States.   

### Datasets and Sources

**1.  Mass Shootings Data from Gun Violence Archive**: The dataset consists of gun violence incidents collected from Jan 01, 2013 to May 18, 2019. Each row of the dataset represents an incident in the United States with details regarding the incident location and number of people killed and injured. Source: https://www.gunviolencearchive.org/

**2. City Locations Data from Opensoft**: The dataset consists of US cities and their geographical locations. Each row represents a city and its geographical coordinates and details like population, timezone, etc. Source: https://public.opendatasoft.com/explore/dataset/1000-largest-us-cities-by-population-with-geographic-coordinates/table/?sort=-rank

**Variables of Interest:**
    <ul>
      <li><i>Incident Date</i>: The date of the mass shooting incident</li>
      <li><i>State</i>: State where the mass shooting incident took place</li>
      <li><i>City</i>: City where the mass shooting incident took place</li>
      <li><i>Address</i>: Address where the mass shooting incident took place</li>
      <li><i>num_Killed</i>: Number of people killed in the mass shooting incident</li>
      <li><i>num_Injured</i>: Number of people injured in the mass shooting incident</li>
      <li><i>Latitude</i>: Latitude of the given city</li>
      <li><i>Longitude</i>: Longitude of the given city</li>
    </ul>
    
**Limitations**
* City Locations Dataset has geolocations only for 1000 largest US cities based on population, so we lose the mass shootings information about some of the smaller cities


In [0]:
import pandas as pd
import numpy as np

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Data Cleaning
The dataset needs some preparation before we start exploring and visualizing the dataset further. Steps to be follwed are:
<ul>
  <li>Renaming columns for efficient data handling</li>
  <li> Creating new features e.g. month and year from date </li>
  <li>Adding latitude and longitude (geolocation) for each of the given locations for maps</li>
  <li>Handling Null values</li>
</ul>

Step1: Reading and merging data for mass shootings from 2013 to 2019

In [0]:
dataset_url = "https://raw.githubusercontent.com/pragyansharma24/Gun-Violence-in-United-States/master/datasets/"

In [4]:
y2013 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2013.csv")
y2014 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2014.csv")
y2015 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2015.csv")
y2016 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2016.csv")
y2017 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2017.csv")
y2018 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2018.csv")
y2019 = pd.read_csv(dataset_url+"Mass%20Shootings%20year%20wise/Mass_Shootings_2019.csv")
# combining data for all the years 
mass_shoot = pd.concat([y2013, y2014, y2015, y2016, y2017, y2018, y2019])
mass_shoot.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,"December 31, 2013",New York,Brooklyn,60 Glenmore Ave,0.0,6.0,NaN
1,"December 28, 2013",Alabama,Montgomery,954 Highland Ave,3.0,5.0,NaN
2,"December 26, 2013",Louisiana,Slidell,2144 First St,2.0,6.0,NaN
3,"December 26, 2013",Louisiana,Lockport,313 Tenth St,3.0,3.0,NaN
4,"December 25, 2013",New Jersey,Irvington,Nye Avenue and 21st Street,3.0,2.0,NaN


Step 2: Renaming columns for efficient data handling

In [0]:
mass_shoot.rename(columns={'Incident Date':'Date','City Or County':'City', '# Killed':'num_Killed', '# Injured':'num_Injured'}, inplace=True)

Step 3: Extracting Month and Year from the incident Date column

In [0]:
from datetime import datetime as dt
mass_shoot['Date'] = pd.to_datetime(mass_shoot['Date'])
mass_shoot['Month'] = mass_shoot['Date'].dt.month
mass_shoot['Year'] = mass_shoot['Date'].dt.year

Step 4: Reading data for Latitudes and Longitudes for cities in the United States

In [7]:
cities_location = pd.read_csv(dataset_url+ "uscitiesv1.5.csv")
# Selecting relevant columns for further analysis
cities_location_sub = cities_location[['city','state_id', 'state_name','lat', 'lng']]
cities_location_sub.head()

,city,state_id,state_name,lat,lng
0,Prairie Ridge,WA,Washington,47.1443,-122.1408
1,Edison,WA,Washington,48.5602,-122.4311
2,Packwood,WA,Washington,46.6085,-121.6702
3,Wautauga Beach,WA,Washington,47.5862,-122.5482
4,Harper,WA,Washington,47.5207,-122.5196


Step 5: Merging mass shootings data with city locations to use with maps. Left join has been used to ensure we have all the data from mass_shoot.


In [0]:
# Merging datasets
mass_shooting_data = mass_shoot.merge(cities_location_sub,how='left', left_on=['City', 'State'], right_on=['city', 'state_name'])
# Dropping extra columns which we won't be using further
mass_shooting_data.drop(columns=['Operations', 'city', 'state_name','state_id'], inplace=True)

Step 6: Removing all the NA values

In [9]:
#Removing NA
mass_shooting_data = mass_shooting_data.dropna()
# Checking number of rows in the combined dataframe
len(mass_shooting_data)

1915

### Final Dataset

In [10]:
mass_shooting_data.head()

,Date,State,City,Address,num_Killed,num_Injured,Month,Year,lat,lng
0,2013-12-31,New York,Brooklyn,60 Glenmore Ave,0.0,6.0,12.0,2013.0,40.6501,-73.9496
1,2013-12-28,Alabama,Montgomery,954 Highland Ave,3.0,5.0,12.0,2013.0,32.3470,-86.2663
2,2013-12-26,Louisiana,Slidell,2144 First St,2.0,6.0,12.0,2013.0,30.2882,-89.7826
3,2013-12-26,Louisiana,Lockport,313 Tenth St,3.0,3.0,12.0,2013.0,29.6418,-90.5376
5,2013-12-25,New York,Medford,33A Cedarhurst Ave,1.0,3.0,12.0,2013.0,40.8220,-72.9859


### Exploration

Number of lives lost in mass shootings by year

In [11]:
mass_shooting_data.groupby('Year')['num_Killed'].sum()\
.reset_index().sort_values(by= 'num_Killed', ascending = False) 

,Year,num_Killed
3,2016.0,411.0
4,2017.0,392.0
2,2015.0,332.0
5,2018.0,328.0
0,2013.0,249.0
1,2014.0,242.0
6,2019.0,130.0


Cities that lost most number of lives in mass shootings

In [12]:
mass_shooting_data.groupby('City')['num_Killed','num_Injured'].sum()\
.reset_index().sort_values(by= 'num_Killed', ascending = False)[:5]                      

,City,num_Killed,num_Injured
106,Chicago,82.0,573.0
285,Las Vegas,72.0,488.0
389,Orlando,68.0,102.0
243,Houston,47.0,102.0
466,Saint Louis,37.0,148.0


In [13]:
mass_shooting_data.groupby('State')['num_Killed','num_Injured'].agg(['sum','count','mean','max'])

num_Killed                  ... num_Injured                  
                            sum count      mean  ...       count       mean    max
State                                            ...                              
Alabama                    46.0    46  1.000000  ...          46   3.978261   12.0
Alaska                      0.0     1  0.000000  ...           1   6.000000    6.0
Arizona                    47.0    27  1.740741  ...          27   3.296296    7.0
Arkansas                   20.0    18  1.111111  ...          18   5.055556   25.0
California                236.0   223  1.058296  ...         223   4.035874   19.0
Colorado                   23.0    21  1.095238  ...          21   3.619048    9.0
Connecticut                 5.0    14  0.357143  ...          14   5.142857   13.0
Delaware                    9.0    10  0.900000  ...          10   3.800000    6.0
District of Columbia        8.0    30  0.266667  ...          30   4.533333   13.0
Florida                   200.0   129  1.550388  ...         129   4.325581   53.0
Georgia                    84.0    76  1.105263  ...          76   3.763158   10.0
Illinois                  119.0   176  0.676136  ...         176   4.210227   12.0
Indiana                    48.0    52  0.923077  ...          52   3.711538    9.0
Iowa                        4.0     5  0.800000  ...           5   3.600000    5.0
Kansas                     30.0    17  1.764706  ...          17   3.823529   14.0
Kentucky                   30.0    26  1.153846  ...          26   3.923077   14.0
Louisiana                  74.0    80  0.925000  ...          80   4.450000   19.0
Maine                       9.0     2  4.500000  ...           2   0.500000    1.0
Maryland                   52.0    52  1.000000  ...          52   3.807692    8.0
Massachusetts               6.0    15  0.400000  ...          15   4.333333    7.0
Michigan                   54.0    60  0.900000  ...          60   4.033333   11.0
Minnesota                  11.0    16  0.687500  ...          16   4.375000    8.0
Mississippi                49.0    32  1.531250  ...          32   3.343750    7.0
Missouri                   64.0    65  0.984615  ...          65   3.676923    8.0
Montana                     9.0     3  3.000000  ...           3   1.666667    3.0
Nebraska                    6.0     7  0.857143  ...           7   4.571429    6.0
Nevada                     72.0    17  4.235294  ...          17  29.235294  441.0
New Jersey                 34.0    54  0.629630  ...          54   4.000000   19.0
New Mexico                 32.0    13  2.461538  ...          13   2.307692    6.0
New York                   46.0    79  0.582278  ...          79   4.202532    9.0
North Carolina             46.0    53  0.867925  ...          53   3.830189    8.0
Ohio                       74.0    66  1.121212  ...          66   3.984848   15.0
Oklahoma                   22.0    21  1.047619  ...          21   3.523810    6.0
Oregon                     15.0     6  2.500000  ...           6   4.333333    9.0
Pennsylvania               76.0    79  0.962025  ...          79   3.924051   11.0
Rhode Island                1.0     4  0.250000  ...           4   4.000000    5.0
South Carolina             50.0    39  1.282051  ...          39   3.717949    8.0
South Dakota               10.0     2  5.000000  ...           2   0.500000    1.0
Tennessee                  54.0    77  0.701299  ...          77   4.064935   10.0
Texas                     198.0   115  1.721739  ...         115   3.573913   18.0
Utah                        9.0     3  3.000000  ...           3   2.333333    6.0
Virginia                   40.0    46  0.869565  ...          46   4.086957   10.0
Washington                 42.0    21  2.000000  ...          21   2.761905    6.0
West Virginia               5.0     3  1.666667  ...           3   3.000000    6.0
Wisconsin                  15.0    14  1.071429  ...          14   3.357143    5.0

[45 rows x 8 columns]

### Scope of Analysis
Combined dataset will be used to analyze the following:
<ul>
  <li><b><i>Mass Shootings across United States:</i></b> Geolocation variables will be used to plot the incidents on a map to visualize the distribution of mass shootings in the United States. This will help us in identifying the states and locations where gun violence is relatively higher.</li>
  <ul>
    <li>Key variable of interest: num_Killed, Latitude, Longitude, City</li>
  </ul>
  <li><b><i>Change in Mass Shooting Incidents over the Years:</i></b> The change in number of mass shooting incidents over the years (2013 - 2019) and/or months across United States will be plotted. This will help us in finding any hidden trends in the data.</li>
  <ul>
    <li>Key variable of interest: Year/Months, num_Killed, num_Injured</li>
    </ul>

*** 

## Crime Statistics across the World

With less than 5% of the world’s population, the United States is home to roughly 35–50 per cent of the world’s civilian-owned guns, heavily skewing the global geography of firearms and any relative comparison. (Ref: https://www.theguardian.com/news/datablog/2012/jul/22/gun-homicides-ownership-world-list)

### Datasets and  Sources
**1. Gun Homicides and Gun Ownership Data from Guardian**: This dataset consists of firearms related murders and ownership details by country. Each row represents a country and details like average firearms per 100 people, total firearms owned by civilians, etc. Source: https://www.theguardian.com/news/datablog/2012/jul/22/gun-homicides-ownership-world-list#data)

**2. Developed Countries 2019 Data from World Population Review**: This dataset is a list of developed countries in the world ranked according to their growth rate and population. Source: http://worldpopulationreview.com/countries/developed-countries/

**Variables of interest**
    <ul>
      <li><i>Country</i>: Name of the country </li>
      <li><i>% of homicides by firearm</i>: Percentage of homicides due to firearms</li>
      <li><i>Homicide by firearm rate per 100,000 pop</i>: Number of homicides happened using firearms per 100,000  </li>
      <li><i>Address</i>: Address where the mass shooting incidence took place</li>
      <li><i>Average total all civilian firearm</i>: Average number of firearms held by civilians</li>
      <li><i>Growth Rate</i>: The rate at which a country is growing according to CIA</li>
    </ul>
    
**Limitations**
<ul>
      <li> Data is present only for Small Arms</li>
      <li> While the website says that the google spreadsheet (Sources: UNODC & Small arms survey) displaying this data is updated every 5 mins, we do not know if the statistics for 'Number of homicides' & 'Average firearms per 100 people' have been updated for the current years or not </li>
      <li> Gun Homicides and Gun Ownership Data for United Kingdom and South Korea is not present</li>
  </ul>


### Data Cleaning

The dataset needs some processing before we start exploring and visualizing the dataset further. Steps to be followed are:
- Renaming column names for efficient data handling
- Handling Null values
- Merging with the list of developed countries

Step 1: Reading in data for world crimes

In [14]:
world_crime = pd.read_csv(dataset_url + "world_crime.csv")
world_crime.head()

,Country/Territory,ISO code,Source,% of homicides by firearm,Number of homicides by firearm,"Homicide by firearm rate per 100,000 pop",Rank by rate of ownership,Average firearms per 100 people,Average total all civilian firearms
0,Albania,AL,CTS,65.9,56.0,1.76,70.0,8.6,270000.0
1,Algeria,DZ,CTS,4.8,20.0,0.06,78.0,7.6,1900000.0
2,Angola,AO,NaN,NaN,NaN,NaN,34.0,17.3,2800000.0
3,Anguilla,AI,WHO-MDB,24.0,1.0,7.14,NaN,NaN,NaN
4,Argentina,AR,Ministry of Justice,52.0,1198.0,3.02,62.0,10.2,3950000.0


Step 2: Renaming columns for efficient data handling

In [0]:
# Creating a new list of column names
cols = ['Country', 'Country_code', 'Source', '%_homi_gun', 'num_homi_gun', 'homi_gun_per100k', 'rank-rate_owner',\
        'avg_gun_per100', 'avg_tot_all_guns']
# Replacing the existing column names with new names saved in the list 'cols'
world_crime.columns = cols

Step 3: Handling Null values

In [16]:
#Checking columns with 'NA' values
world_crime.isna().sum()

Country              0
Country_code         1
Source              69
%_homi_gun          69
num_homi_gun        69
homi_gun_per100k    69
rank-rate_owner      9
avg_gun_per100       9
avg_tot_all_guns     9
dtype: int64

In [17]:
# Removing all the 'NA' values
world_crime = world_crime.dropna()
len(world_crime)

107

Step 4: Merging the crime data will all the countries and combining with a list of developed countries.

In [18]:
# Reading in data for growth rates for all 'Developed' countries
developed_countries = pd.read_csv(dataset_url+"developed.csv")
developed_countries.head()

,cca2,name,area,pop2019,GrowthRate
0,US,United States,9372610,329093.110,1.007119
1,JP,Japan,377930,126854.745,0.997401
2,TR,Turkey,783562,82961.805,1.012756
3,DE,Germany,357114,82438.639,1.001764
4,GB,United Kingdom,242900,66959.016,1.005791


In [19]:
# Checking for NAs
developed_countries.isna().sum()

cca2          0
name          0
area          0
pop2019       0
GrowthRate    0
dtype: int64

Merging world crime data with growth index to the find the position of the United States as compared to other developed countries. We are using inner join because we want to see the data only for developed countries.

In [0]:
# Merging world crimes and growth rates dataset
world_crime_developed = world_crime.merge(developed_countries, how="inner", left_on="Country", right_on="name")
# Removing columns that are not required for the analysis
world_crime_developed.drop(columns = ['Source', 'cca2','name', 'area'], inplace = True)

### Final Dataset

In [21]:
world_crime_developed.head()

,Country,Country_code,%_homi_gun,num_homi_gun,homi_gun_per100k,rank-rate_owner,avg_gun_per100,avg_tot_all_guns,pop2019,GrowthRate
0,Australia,AU,11.5,30.0,0.14,42.0,15.0,3050000.0,25088.636,1.012772
1,Austria,AT,29.5,18.0,0.22,14.0,30.4,2500000.0,8766.201,1.001643
2,Belgium,BE,39.5,70.0,0.68,34.0,17.2,1800000.0,11562.784,1.005589
3,Canada,CA,32.0,173.0,0.51,13.0,30.8,9950000.0,37279.811,1.008823
4,Cyprus,CY,26.3,5.0,0.46,6.0,36.4,275000.0,1198.427,1.007856


### Exploration

Countries as per gun ownership rate

In [22]:
world_crime_developed.sort_values(by = 'rank-rate_owner')[:5]

,Country,Country_code,%_homi_gun,num_homi_gun,homi_gun_per100k,rank-rate_owner,avg_gun_per100,avg_tot_all_guns,pop2019,GrowthRate
28,United States,US,60.0,9146.0,2.97,1.0,88.8,270000000.0,329093.110,1.007119
26,Switzerland,CH,72.2,57.0,0.77,3.0,45.7,3400000.0,8608.259,1.007517
8,Finland,FI,19.8,24.0,0.45,4.0,45.3,2400000.0,5561.389,1.003405
4,Cyprus,CY,26.3,5.0,0.46,6.0,36.4,275000.0,1198.427,1.007856
25,Sweden,SE,33.9,37.0,0.41,10.0,31.6,2800000.0,10053.135,1.007055


Countries by average number of average total all civilian firearms

In [33]:
world_crime_developed.sort_values(by = 'avg_tot_all_guns', ascending = False)[:5]

,Country,Country_code,%_homi_gun,num_homi_gun,homi_gun_per100k,rank-rate_owner,avg_gun_per100,avg_tot_all_guns,pop2019,GrowthRate
28,United States,US,60.0,9146.0,2.97,1.0,88.8,270000000.0,329093.110,1.007119
10,Germany,DE,26.3,158.0,0.19,15.0,30.3,25000000.0,82438.639,1.001764
9,France,FR,9.6,35.0,0.06,12.0,31.2,19000000.0,65480.710,1.003793
3,Canada,CA,32.0,173.0,0.51,13.0,30.8,9950000.0,37279.811,1.008823
27,Turkey,TR,16.9,535.0,0.77,52.0,12.5,9000000.0,82961.805,1.012756


### Scope of Analysis
The combined dataset can be used to analyze the following: 

<ul>
  <li><b><i>Homicides by firearm per 1 million people: </i></b>
This visualization will showcase developed countries according to firearms related homicides. The argument we are trying to present here is that gun availability is a major risk factor for homicides in developed nations. </li>
  <ul> <li>Key variables of interest - Country, homi_gun_per100k </li> </ul>
<li>
  <b><i>Population vs Ownership of firearms: </i></b> In this graph we will plot the population of various countries as well as the ownership of firearms in those countries. This visualization will show us a comparison of the number of people living in a country versus the number of civilians who own firearms in those countries. </li>
  <ul> <li>Key variables of interest: Country, pop2019, avg_tot_all_guns </li></ul>
  
  <li>
    <b><i>Gun related deaths per 100k people vs guns per 100 people: </i></b> This visualization will be a scatter plot comparing gun related homicides against gun possession across developed countries. This visualization will help us identify relationship between gun ownership and gun related deaths across developed nations.
    <ul><li>Key variables of interest: Country, homi_gun_per100k, avg_gun_per100</li></ul>
  </li>

---

## Presence of Gun Laws and Death Rates across United States


On an average 10 out of every 100,000 people are killed by firearms across the United States. And, it is commonly assumed that mental illness or stress levels are few of the major triggers. But, what about gun control laws? Might tighter gun control laws make a difference? Some studies indicate that they do.

Therefore, in this section, we are trying to analyze the relation of firearm related deaths with gun control restrictions by state. It includes states which have one of three gun control restrictions in place - assault weapons' bans, trigger locks, or safe storage requirements. (Source: https://www.theatlantic.com/national/archive/2011/01/the-geography-of-gun-deaths/69354/).

### Datasets and Sources
*1. Firearms Child Protection*: This dataset consists details of states with firearm laws designed to protect children. Each row represents whether, 'Child Access Prevention Firearm Law', 'Safe Storage Gun Lock Requirment' and 'Assault Weapons Ban' is present or not. (Source: https://www.kff.org/other/state-indicator/firearms-and-children-legislation/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

*2. Firearms Deaths*: This dataset consists number of deaths due to injury by firearms per 100,000 population in each state. (Source: https://www.kff.org/other/state-indicator/firearms-death-rate-per-100000/?currentTimeframe=3&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)


**Variables of Interest:**
    <ul>
      <li><i>Child Access Prevention Firearm Law</i>: State laws in place designed to prevent children from accessing firearms </li>
      <li><i>Safe Storage or Gun Lock Requirment</i>: State laws concerning firearms locking devices in place</li>
      <li><i>Assault Weapons Ban</i>: State law banning assault weapons, by name, or by specific features of the gun that make a gun an assault weapon in place or not</li>
      <li><i>Firearms Death Rate per 100,000</i>: Deaths rate to injury by firearms per 100,000 Population</li>
    </ul>
    
**Limitations**
 *  Firearms Child Protection dataset is only present for 2014 so the analysis is done only on 2014 data. Some states might have implemented laws which could not be captured
 * State laws applies to assault pistols only
 * Deaths from injury due to firearms excludes deaths due to explosives and other causes indirectly caused by firearms.

### Data Cleaning

The dataset needs some processing before we start exploring and visualizing the dataset further. Steps to be followed are:
- Merging  'Firearms Deaths' data with  'Firearm Child Protection'
- Creating new columns for states which have at least one gun law present

Step 1: Reading  the datset for firearms and child protection

In [0]:
firearm_child_protection = pd.read_csv(dataset_url + 'firearm_children.csv', skiprows = 2)

Step 2: Removing columns and rows that are not required for the analysis

In [0]:
firearm_child_protection= firearm_child_protection[~(firearm_child_protection.Location == 'United States')]
firearm_child_protection.drop(columns=['Footnotes'], inplace = True)

Step 3: Creating a new column atleast_1_law which has any of the laws implemented Child Access Prevention Firearm Law, Safe Storage or Gun Lock Requirment or Assault Weapons Ban

In [0]:
firearm_child_protection['atleast_1_law']=((firearm_child_protection['Child Access Prevention Firearm Law']=='Yes')|
                                                    (firearm_child_protection['Safe Storage or Gun Lock Requirment']=='Yes')|
                                                    (firearm_child_protection['Assault Weapons Ban']=='Yes'))*1

Step 4: Reading  the datset for firearms and death rate

In [0]:
firearm_death = pd.read_csv(dataset_url +'firearm_death.csv', skiprows = 2)

Step 5: Removing columns and rows that are not required for the analysis

In [0]:
firearm_death= firearm_death[~(firearm_death.Location == 'United States')]
firearm_death= firearm_death[~(firearm_death.Location == 'United States')]

Step 6: Merging firearms child protection with firearms death data using inner join on state as we need information from both the datasets for our analysis

In [0]:
firearm_statelaws_child_protection = firearm_death.merge(firearm_child_protection, how="inner", left_on="Location", right_on="Location")

### Final Dataset

In [31]:
firearm_statelaws_child_protection.head()

,Location,"Firearms Death Rate per 100,000",Child Access Prevention Firearm Law,Safe Storage or Gun Lock Requirment,Assault Weapons Ban,atleast_1_law
0,Alabama,16.9,No,No,No,0
1,Alaska,19.2,No,No,No,0
2,Arizona,13.5,No,No,No,0
3,Arkansas,16.6,No,No,No,0
4,California,7.4,Yes,Yes,Yes,1


### Exploration

Average death rate related to firearms when the gun laws are present vs not present

In [32]:
firearm_statelaws_child_protection.groupby('Child Access Prevention Firearm Law')['Firearms Death Rate per 100,000'].agg(['mean'])
firearm_statelaws_child_protection.groupby('Safe Storage or Gun Lock Requirment')['Firearms Death Rate per 100,000'].agg(['mean'])
firearm_statelaws_child_protection.groupby('Assault Weapons Ban')['Firearms Death Rate per 100,000'].agg(['mean'])
firearm_statelaws_child_protection.groupby('atleast_1_law')['Firearms Death Rate per 100,000'].agg(['mean'])

,mean
Child Access Prevention Firearm Law,
No,12.930435
Yes,10.228571


,mean
Safe Storage or Gun Lock Requirment,
No,12.645000
Yes,7.090909


,mean
Assault Weapons Ban,
No,12.451163
Yes,6.050000


,mean
atleast_1_law,
0,13.752632
1,10.078125


### Scope of Analysis
Combined dataset will be used to analyze the following:
<ul>
  <li><b><i>Relation of gun restriction laws with gun related deaths :</i></b> This visualization will showcase the relation of firearm deaths with gun control restrictions for all US states. This will help us analyze if firearm deaths are significantly lower in states with stricter gun control legislation.</li>
  <ul>
    <li>Key variable of interest: Assault Weapons Ban, atleast_1_law, Child Access Prevention Firearm Law, Safe Storage or Gun Lock Requirment, Firearms Death Rate per 100,000</li>
  </ul>

*** 

## References

* America’s unique gun violence problem, explained in 17 maps and charts 
https://www.vox.com/policy-and-politics/2017/10/2/16399418/us-gun-violence-statistics-maps-charts

* Gun homicides and gun ownership listed by country 
https://www.theguardian.com/news/datablog/2012/jul/22/gun-homicides-ownership-world-list#data

* Developed Countries 2019
http://worldpopulationreview.com/countries/developed-countries/

* Gun Violence Archive (GVA) 
https://www.gunviolencearchive.org/

* 1000 Largest US Cities By Population With Geographic Coordinates
https://public.opendatasoft.com/explore/dataset/1000-largest-us-cities-by-population-with-geographic-coordinates/table/?sort=-rank

* Harvard Injury Control Research Center
https://www.hsph.harvard.edu/hicrc/firearms-research/guns-and-death/

* States with Firearm Laws Designed to Protect Children
https://www.kff.org/other/state-indicator/firearms-and-children-legislation/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

* Number of Deaths Due to Injury by Firearms per 100,000 Population
https://www.kff.org/other/state-indicator/firearms-death-rate-per-100000/?currentTimeframe=3&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

* Nearly 40,000 People Died From Guns in U.S. Last Year, Highest in 50 Years
https://www.nytimes.com/2018/12/18/us/gun-deaths.html

* The Geography of Gun Deaths https://www.theatlantic.com/national/archive/2011/01/the-geography-of-gun-deaths/69354/
